In [ ]:
# Retrive closest match embeddings from Pinecone and send context to Open AI completion endpoint to build answer
import pinecone
import openai

PINECONE_API_KEY = ''
PINECONE_ENV = ''
PINECONE_INDEX_NAME = ''
PINECONE_NAMESPACE = ''

OPEN_AI_API_KEY = ''
EMBEDDING_MODEL = ''

def retrieve_context(query):
    pinecone.init(
        api_key=PINECONE_API_KEY,
        environment=PINECONE_ENV,
    )

    index = pinecone.Index(PINECONE_INDEX_NAME)

    openai.api_key = OPEN_AI_API_KEY

    res = openai.Embedding.create(
        input=[query],
        engine=EMBEDDING_MODEL
    )

    xq = res['data'][0]['embedding']
    res = index.query(xq, top_k=3, include_metadata=True, namespace=PINECONE_NAMESPACE)

    contexts = [
        x['metadata']['text'] for x in res['matches']
    ]

    return contexts

def build_prompt(query, context):
    context_limit = 3750
    prompt = ""
    prompt_start = (
        "You are a meeting log retrieval bot. You receive questions about a library of project logs and return answers.\n\n" +
        "Remember to:\n\n" +
        "- Format responses with markdown.\n\n" + 
        "- Look for days in the format Month, Day, Year. For example, May 13, 2023.\n\n" +
        "- Only return answers from th provided content below:.\n\n"
    )
    
    prompt_end = (
        f"\n\nQuestion: {query}\nAnswer:"
    )
       
    for i in range(1, len(context)):
        if len("\n\n---\n\n".join(context[:i])) >= context_limit:
            prompt = (
                prompt_start +
                "\n\n---\n\n".join(context[:i-1]) +
                prompt_end
            )
            break
        elif i == len(context)-1:
            prompt = (
                prompt_start +
                "\n\n---\n\n".join(context) +
                prompt_end
            )

    return prompt

def complete(prompt):
    res = openai.Completion.create(
        engine='text-davinci-003',
        prompt=prompt,
        temperature=0.2,
        max_tokens=2000,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )
    return res['choices'][0]['text'].strip()

def answer(query):
    context = retrieve_context(query)
    prompt = build_prompt(query, context)
    answer = complete(prompt)
    
    print(answer)

answer("")